In [1]:
from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.aqua import run_algorithm
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.aqua.translators.ising import docplex

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  
# choose INFO, DEBUG to see the log
import numpy as np

In [9]:
# Create an instance of a model and variables
mdl   = Model(name = 'Logistics')
comb  = [(1, 11), (1, 12), (2, 11), (2, 12), 
         (11, 101), (11, 102), (12, 101), (12, 102)]
x     = {(i, j): mdl.binary_var(name = 'x_' + str((i, j))) 
                for (i, j) in comb}
costs = {(i, j): np.random.randint(1, 5)
                for (i, j) in comb}

In [10]:
# Objective function
max_vars_func = mdl.sum(costs[i, j]*x[i, j] 
                        for (i, j) in comb)
mdl.minimize(max_vars_func)

# Constraints

#Final demand
for j in [101, 102]:
    mdl.add_constraint(mdl.sum(x[i, j] for i in 
            set([i for (i, l) in comb if l == j])) == 1)

#Transportation
for j in [11, 12]:
    mdl.add_constraint(mdl.sum(x[i, j] for i in 
            set([i for (i, l) in comb if l == j])) ==
            mdl.sum(x[j, k] for k in 
            set([k for (l, k) in comb if l == j])))
    
#Source
for j in [11, 12]:
    mdl.add_constraint(mdl.sum(x[i, j] for i in 
            set([i for (i, l) in comb if l == j])) == 1)

print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Logistics

Minimize
 obj: x_(1,_11) + 3 x_(1,_12) + 4 x_(2,_11) + 4 x_(2,_12) + 2 x_(11,_101)
      + 4 x_(11,_102) + 4 x_(12,_101) + 4 x_(12,_102)
Subject To
 c1: x_(11,_101) + x_(12,_101) = 1
 c2: x_(11,_102) + x_(12,_102) = 1
 c3: x_(1,_11) + x_(2,_11) - x_(11,_101) - x_(11,_102) = 0
 c4: x_(1,_12) + x_(2,_12) - x_(12,_101) - x_(12,_102) = 0
 c5: x_(1,_11) + x_(2,_11) = 1
 c6: x_(1,_12) + x_(2,_12) = 1

Bounds
 0 <= x_(1,_11) <= 1
 0 <= x_(1,_12) <= 1
 0 <= x_(2,_11) <= 1
 0 <= x_(2,_12) <= 1
 0 <= x_(11,_101) <= 1
 0 <= x_(11,_102) <= 1
 0 <= x_(12,_101) <= 1
 0 <= x_(12,_102) <= 1

Binaries
 x_(1,_11) x_(1,_12) x_(2,_11) x_(2,_12) x_(11,_101) x_(11,_102) x_(12,_101)
 x_(12,_102)
End



In [11]:
qubitOp, offset = docplex.get_qubitops(mdl)

In [16]:
ee = ExactEigensolver(qubitOp, k = 4)
result = ee.run()

print('energy:', result['energy'])
print('objective:', result['energy'] + offset)

x = docplex.sample_most_likely(result['eigvecs'][0])
print('solution:', x)

energy: -88.0
objective: 60.0
solution: [0. 0. 0. 0. 1. 0. 0. 1.]


In [7]:
spsa             = SPSA(max_trials = 300)
ry               = RY(qubitOp.num_qubits, depth = 5, entanglement = 'linear')
vqe              = VQE(qubitOp, ry, spsa)

backend          = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

result           = vqe.run(quantum_instance)

In [8]:
x = docplex.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution objective:', result['energy'] + offset)
print('solution:', x)

energy: -91.15441727751285
time: 348.4387638568878
solution objective: 2.8455827224871513
solution: [0. 0. 0. 0. 1. 0. 0. 1.]
